# Approximate single value decomposition of a video dataset

The aproximate implementation works by:
* binned the movie in 1 s bins (per channel)
* Estimating U from the binned movie.
* Iterating over the raw data to estimate S.VT 

Details in wfield-tools/decomposition.py 

The first example subtracts the average from each frame (i.e. the array in `frames_average` with 3 dimensions (ch,w,h)). The average can be the mean of all frames or only those in the baseline period of the experiment.


In [13]:
from wfield import *

datadisk = (r'/datajoint-data/data/aeltona/AA_ROS-1706_2024-03-12_scan9FN2ANVG_sess9FN2ANVG/tif files')
# dat = load_stack(r'D:\data\aeltona\AA_ROS-1706_2024-03-12_scan9FN2ANVG_sess9FN2ANVG/scan9FN2ANVG_Oddball_AA_ROS-1706_2025_MMStack_Default.ome.tif')


In [16]:
glob(pjoin(datadisk, '*.tif'))[1]

'/datajoint-data/data/aeltona/AA_ROS-1706_2024-03-12_scan9FN2ANVG_sess9FN2ANVG/tif files/scan9FN2ANVG_Oddball_AA_ROS-1706_2025_MMStack_Default.ome.tif'

In [17]:
# dat = load_stack(glob(pjoin(datadisk, '*.tif'))[0])
dat = load_stack(r'/datajoint-data/data/aeltona/AA_ROS-1706_2024-03-12_scan9FN2ANVG_sess9FN2ANVG/tif files/scan9FN2ANVG_Oddball_AA_ROS-1706_2025_MMStack_Default.ome.tif')
# dat

Parsing tiffs:   0%|          | 0/1 [00:00<?, ?it/s]<tifffile.TiffFile 'scan9FN2ANVG_Od…_Default.ome.tif'> MMStack file name is invalid
<tifffile.TiffFile 'scan9FN2ANVG_Od…_Default.ome.tif'> MMStack series is missing files. Returning subset (5906, 1, 1, 1) of (18000, 1, 1, 1)
Parsing tiffs:   0%|          | 0/1 [00:00<?, ?it/s]


ValueError: image data are not memory-mappable

In [8]:
import matplotlib
matplotlib.use('Agg')

In [12]:
# %matplotlib 
nb_play_movie(dat)

AttributeError: 'str' object has no attribute 'shape'

In [ ]:
dat = load_stack(filename)
# frames_average = np.load('frames_average.npy')
chunkidx = chunk_indices(len(dat),chunksize=256)
frame_averages = []
for on,off in tqdm(chunkidx, desc='Computing average.'):
    frame_averages.append(dat[on:off].mean(axis = 0))
frames_average = np.stack(frame_averages).mean(axis = 0)
U,SVT = approximate_svd(dat, frames_average,nframes_per_bin=2)


In [11]:
np.save(pjoin(localdisk,'U.npy'),U)
np.save(pjoin(localdisk,'SVT.npy'),SVT)

In [7]:
localdisk = r'D:\data\aeltona\AA_ROS-1706_2024-03-12_scan9FN2ANVG_sess9FN2ANVG'

In [8]:
# Reconstruct and reshape into 2 channels
onset = 100
nframes = 10
mov = reconstruct(U,SVT[:,onset:onset+nframes]
                 ).reshape(nframes,1,*U.shape[:2])
# take a look at the differerce between the raw and the reconstructed
%matplotlib notebook

nb_play_movie((mov*frames_average)+frames_average)# - dat[onset:onset+nframes])

ValueError: cannot reshape array of size 2000 into shape (600,600,newaxis)

In [10]:
SVT.shape

(200, 5906)